# Lecture 10 — Classes, Methods, and Subclasses (Python)

This notebook accompanies the attached slide deck.

## Learning goals
By the end of this notebook, you should be able to:
- Define a class and create objects (instances).
- Use `__init__` to initialize attributes reliably.
- Write instance methods and explain the role of `self`.
- Customize string output with `__repr__` / `__str__`.
- Create subclasses, call superclass initializers with `super()`, and reason about method lookup.
- Distinguish `is` vs `==`, and implement custom equality with `__eq__`.


In [ ]:
# Imports used in examples
import math


## 1) Classes and objects (instances)

A **class** defines a new type. An **object** is an instance of that class.

From the slides: a minimal class for 3D points:


In [ ]:
class Point3:
    pass

p = Point3()
p.x = 2
p.y = 3
p.z = 5

p

Manual attribute setup is easy to get wrong (misspellings, missing attributes, wrong types).  
Use an initializer (`__init__`) to make object creation consistent.


## 2) Constructors and initializers (`__init__`)

Calling `Point3(2, 3, 5)` triggers the object-creation protocol:
1. Allocate a new object.
2. Call `__init__(self, ...)` to initialize it.
3. Return the new object.

A conventional first parameter name is **`self`**.


In [ ]:
class Point3:
    def __init__(self, x_val, y_val, z_val):
        self.x = x_val
        self.y = y_val
        self.z = z_val

point1 = Point3(2, 3, 5)
(point1.x, point1.y, point1.z)

Example from the slides: a social media post with a counter-like field.


In [ ]:
class Post:
    def __init__(self, text):
        """A post whose text is `text` and with no likes so far."""
        self.text = text
        self.n_likes = 0

post1 = Post("good vibes")
(post1.text, post1.n_likes)

## 3) Methods and the `self` parameter

Methods are functions stored in the class, but called on objects.
- Methods use `self` to access the receiver object's attributes.
- A call like `ctr.currCount()` implicitly passes the object as the first argument.


In [ ]:
class Counter:
    def __init__(self):
        self.count = 0
    def currCount(self):
        return self.count
    def incr(self):
        self.count += 1
    def reset(self):
        self.count = 0

ctr = Counter()
ctr.currCount(), (ctr.incr() or None), ctr.currCount()

Equivalently, `ctr.currCount()` can be viewed as `Counter.currCount(ctr)`.


In [ ]:
Counter.currCount(ctr)

**Common error:** forgetting `self` in a method header causes an argument mismatch at call time.


In [ ]:
class BadCounter:
    def __init__(self):
        self.count = 0
    # BUG: missing `self`
    def currCount():
        return 0

bc = BadCounter()
try:
    bc.currCount()
except TypeError as e:
    print("TypeError:", e)

## 4) String representations: `__repr__` (and `__str__`)

In interactive mode, Python typically shows objects using a representation string.
Defining `__repr__` lets you display custom objects nicely.


In [ ]:
class Point3:
    def __init__(self, x, y, z):
        self.x, self.y, self.z = x, y, z

    def __repr__(self):
        return "Point3(" + str(self.x) + ", " + str(self.y) + ", " + str(self.z) + ")"

Point3(2, 3, 5)

Add a normal (non-dunder) method: Euclidean distance between two 3D points.


In [ ]:
class Point3:
    def __init__(self, x, y, z):
        self.x, self.y, self.z = x, y, z

    def __repr__(self):
        return f"Point3({self.x}, {self.y}, {self.z})"

    def distance(self, other):
        """Euclidean distance between two points."""
        xd = (other.x - self.x) ** 2
        yd = (other.y - self.y) ** 2
        zd = (other.z - self.z) ** 2
        return math.sqrt(xd + yd + zd)

p = Point3(1, 2, 3)
p0 = Point3(0, 0, 0)
p.distance(p0)

## 5) Subclassing and inheritance

A **subclass** extends a **superclass** by adding or customizing behavior.
Python searches for attributes/methods using the *bottom-up rule*:
1. Look in the object's class.
2. If not found, look in its superclass chain.
3. Ultimately, everything inherits from `object`.


In [ ]:
class Account:
    def __init__(self, owner, number):
        self._owner = owner
        self._number = number
        self._balance = 0.0

    def deposit(self, amount):
        self._balance += amount

    def withdraw(self, amount):
        self._balance -= amount

    def balance(self):
        return self._balance

Subclass example from the slides: initialize superclass attributes using `super().__init__(...)`, then add subclass-specific fields.


In [ ]:
class CreditAccount(Account):
    def __init__(self, owner, number, limit):
        super().__init__(owner, number)   # initialize Account part
        self._limit = limit               # subclass-specific attribute

    def available_credit(self):
        return self._limit + self._balance  # balance can be negative after withdrawals

ca = CreditAccount("Bean", "274563784", limit=5000.0)
ca.deposit(1000.0)
ca.withdraw(1200.0)
(ca.balance(), ca.available_credit())

## 6) Type testing: `isinstance` vs `type(...) == ...`

- `isinstance(x, C)` is true if `x` is an instance of `C` **or any subclass** of `C`.
- `type(x) == C` is true only if `x`'s class is exactly `C`.


In [ ]:
class DepositAccount(Account):
    pass

class InterestAccount(DepositAccount):
    pass

a = InterestAccount("Bean", "274563784")

print("isinstance(a, InterestAccount):", isinstance(a, InterestAccount))
print("isinstance(a, DepositAccount): ", isinstance(a, DepositAccount))
print("isinstance(a, Account):        ", isinstance(a, Account))
print("isinstance(a, object):         ", isinstance(a, object))
print("isinstance(a, CreditAccount):  ", isinstance(a, CreditAccount))

print("type(a) == InterestAccount:", type(a) == InterestAccount)
print("type(a) == Account:       ", type(a) == Account)

## 7) Method overriding

If a subclass defines a method that already exists in the superclass, the subclass version is used first (bottom-up rule).


In [ ]:
class Account:
    def __init__(self, owner, number):
        self._owner = owner
        self._number = number
        self._balance = 0.0

    def __str__(self):
        return "Account #" + str(self._number)

class CreditAccount(Account):
    def __init__(self, owner, number, limit):
        super().__init__(owner, number)
        self._limit = limit

    # override __str__
    def __str__(self):
        return "CreditAccount #" + str(self._number) + f" (limit={self._limit})"

print(str(Account("A", "001")))
print(str(CreditAccount("B", "002", 5000)))

## 8) Identity vs equality (`is` vs `==`) and `__eq__`

- `is` checks whether two variables refer to the **same object** (identity).
- `==` checks **equivalence**, which can be customized by overriding `__eq__`.

By default, many user-defined classes behave like `object.__eq__`, which is essentially identity-based.


In [ ]:
def incr(n): 
    return n + 1

x = 1000
y = incr(x) - 1

print("x == y:", x == y)
print("x is y:", x is y)  # integers may be interned in some cases; do not rely on identity for numbers

Custom equivalence for `Point3`: two points are equal if their coordinates match.

Note: the slides use `type(other) == Point3` (not `isinstance`) to restrict equality to the exact class.


In [ ]:
class Point3:
    def __init__(self, x, y, z):
        self.x, self.y, self.z = x, y, z

    def __repr__(self):
        return f"Point3({self.x}, {self.y}, {self.z})"

    def __eq__(self, other):
        """Same coordinates (exact type match)."""
        return type(other) == Point3 and self.x == other.x and self.y == other.y and self.z == other.z

print(Point3(0,0,0) == Point3(0,0,0))
print(Point3(0,0,0) is Point3(0,0,0))

## 9) Practice (exercises)

### Exercise A — Add a method
Add a method `translate(self, dx, dy, dz)` to `Point3` that shifts the point in-place.

### Exercise B — Add `__str__`
Implement `__str__` for `Point3` to return a user-friendly string, e.g. `"(x, y, z)"`.

### Exercise C — Subclass behavior
Create a subclass `FeeAccount(Account)` that charges a fixed fee on each withdrawal:
- It should call `super().withdraw(amount + fee)`.

Work in the code cell below.


In [ ]:
# TODO: solve the exercises here

# (A) translate
# (B) __str__
# (C) FeeAccount



### Quick self-check tests (optional)

Uncomment and run after you implement the exercises.


In [ ]:
# # Self-checks
# p = Point3(1,2,3)
# p.translate(10, 0, -1)
# assert (p.x, p.y, p.z) == (11,2,2)
# print(str(p))  # should be user-friendly if you implemented __str__
#
# fa = FeeAccount("C", "003", fee=2.5)
# fa.deposit(100.0)
# fa.withdraw(10.0)
# assert abs(fa.balance() - 87.5) < 1e-9  # 100 - (10+2.5)
# print("All checks passed.")
